In [ ]:
# In order to run these codes (parsing pdfs step 1 and 2), you can run them in a Google Colab notebook.
# To run fast, you should get connected to a GPU in the Colab environment.
# In the Colab secrets, create and copy your HUGGING_FACE_TOKEN and MONGO_URI (MongoDB URI) to the Secrets section.

In [ ]:
import pandas as pd
from ast import literal_eval

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/My Drive/Colab Notebooks/pdfs/TheOnlyCancerPatientCure_PDF_Download.pdf'

Mounted at /content/drive


In [ ]:
pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.3 MB/s eta 0:00:00


In [ ]:
import PyPDF2

def extract_sentences(pdf_file):
    with open(pdf_file, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()

    sentences = text.split('. ') # Basic sentence splitting
    return sentences

sentences = extract_sentences(file_path)


In [ ]:
sentences_endline_removed = pd.Series(sentences).str.replace('\n', '')

In [ ]:
from sentence_transformers import SentenceTransformer

# https://huggingface.co/thenlper/gte-large
embedding_model = SentenceTransformer("thenlper/gte-large")


def get_embedding(text: str) -> list[float]:
    if not text.strip():
        print("Attempted to get embedding for empty text.")
        return []

    embedding = embedding_model.encode(text)

    return embedding.tolist()


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.9k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [ ]:
# Uncomment for new runs if needed:
dataset_embedded = pd.Series(sentences_endline_removed).apply(get_embedding)

In [ ]:
dataset = pd.DataFrame(dataset_embedded)
dataset.rename({0: 'embedding_'}, axis=1, inplace=True)

In [ ]:
#dataset_new = pd.DataFrame(dataset)
dataset['sentences'] = sentences_endline_removed

In [ ]:
# save the csv file:
file_path_to_save = '/content/drive/My Drive/Colab Notebooks/csv_saved/'
dataset.to_csv(file_path_to_save+'dataset_embedded.csv', index=False)
